# 05. 短時間フーリエ変換（STFT）

時間と周波数の両方を同時に分析するSTFTについて学びます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. STFTの基本概念

In [ ]:
def manual_stft(x, window_size, overlap, window_type='hann', sample_rate=44100):
    """
    手動でSTFTを実装（理解のため）
    """
    hop_size = window_size - overlap
    
    # 窓関数の作成
    if window_type == 'hann':
        window = np.hanning(window_size)
    elif window_type == 'hamming':
        window = np.hamming(window_size)
    elif window_type == 'rectangular':
        window = np.ones(window_size)
    
    # 時間軸とフレーム数の計算
    num_frames = (len(x) - window_size) // hop_size + 1
    
    # STFT結果を格納する配列
    stft_result = np.zeros((window_size // 2 + 1, num_frames), dtype=complex)
    
    # 各フレームでFFT
    for frame in range(num_frames):
        start = frame * hop_size
        end = start + window_size
        
        # 窓関数を適用
        windowed_signal = x[start:end] * window
        
        # FFT
        fft_result = fft(windowed_signal)
        
        # 正の周波数のみ保存
        stft_result[:, frame] = fft_result[:window_size // 2 + 1]
    
    # 時間軸と周波数軸
    times = np.arange(num_frames) * hop_size / sample_rate
    frequencies = np.fft.fftfreq(window_size, 1/sample_rate)[:window_size // 2 + 1]
    
    return frequencies, times, stft_result

# 時間とともに周波数が変化する信号（チャープ）
sample_rate = 8000
duration = 3.0
t = np.linspace(0, duration, int(sample_rate * duration), False)

# チャープ信号（100Hz → 1000Hz）
chirp_signal = signal.chirp(t, 100, duration, 1000, method='linear')

# 複数音が混在する信号
multi_tone = (np.sin(2 * np.pi * 200 * t) * (t < 1) +
             np.sin(2 * np.pi * 500 * t) * ((t >= 1) & (t < 2)) +
             np.sin(2 * np.pi * 800 * t) * (t >= 2))

# 窓サイズの影響を比較
window_sizes = [128, 256, 512, 1024]

plt.figure(figsize=(18, 12))

# チャープ信号のSTFT比較
for i, win_size in enumerate(window_sizes):
    # 手動STFT
    freqs, times, stft_result = manual_stft(chirp_signal, win_size, win_size//2, 'hann', sample_rate)
    
    plt.subplot(2, 4, i + 1)
    magnitude = np.abs(stft_result)
    plt.pcolormesh(times, freqs, 20 * np.log10(magnitude + 1e-10), 
                  shading='gouraud', cmap='viridis')
    plt.title(f'チャープ信号\n窓サイズ: {win_size}')
    plt.ylabel('周波数 (Hz)')
    plt.ylim(0, 1500)
    
    # 時間分解能と周波数分解能を表示
    time_res = (win_size // 2) / sample_rate * 1000  # ms
    freq_res = sample_rate / win_size  # Hz
    plt.text(0.1, 1300, f'時間分解能: {time_res:.1f} ms\n周波数分解能: {freq_res:.1f} Hz', 
            bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
            fontsize=9)

# マルチトーン信号のSTFT比較
for i, win_size in enumerate(window_sizes):
    freqs, times, stft_result = manual_stft(multi_tone, win_size, win_size//2, 'hann', sample_rate)
    
    plt.subplot(2, 4, i + 5)
    magnitude = np.abs(stft_result)
    plt.pcolormesh(times, times, 20 * np.log10(magnitude + 1e-10), 
                  shading='gouraud', cmap='plasma')
    plt.title(f'マルチトーン信号\n窓サイズ: {win_size}')
    plt.xlabel('時間 (秒)')
    plt.ylabel('周波数 (Hz)')
    plt.ylim(0, 1000)

plt.tight_layout()
plt.show()

# 元の信号も表示
plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.plot(t, chirp_signal)
plt.title('チャープ信号（100Hz → 1000Hz）')
plt.ylabel('振幅')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(t, multi_tone)
plt.title('マルチトーン信号（200Hz → 500Hz → 800Hz）')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.grid(True)

plt.tight_layout()
plt.show()

print("窓サイズの影響:")
print("- 小さい窓: 高い時間分解能、低い周波数分解能")
print("- 大きい窓: 低い時間分解能、高い周波数分解能")
print("- トレードオフの関係（不確定性原理）")

## 2. 音楽信号のSTFT解析

In [ ]:
# 音楽的な信号の生成
def create_musical_sequence(sample_rate, duration_per_note=0.5):
    """
    音楽的なシーケンスを生成
    """
    # C Major Scale (ド レ ミ ファ ソ ラ シ ド)
    notes = [261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88, 523.25]  # Hz
    
    total_duration = len(notes) * duration_per_note
    t_total = np.linspace(0, total_duration, int(sample_rate * total_duration), False)
    
    music_signal = np.zeros_like(t_total)
    
    for i, note_freq in enumerate(notes):
        start_time = i * duration_per_note
        end_time = (i + 1) * duration_per_note
        
        start_idx = int(start_time * sample_rate)
        end_idx = int(end_time * sample_rate)
        
        # ノートの時間軸
        note_t = t_total[start_idx:end_idx] - start_time
        
        # 楽器的な音（倍音を含む）
        note_signal = (np.sin(2 * np.pi * note_freq * note_t) +
                      0.5 * np.sin(2 * np.pi * note_freq * 2 * note_t) +
                      0.3 * np.sin(2 * np.pi * note_freq * 3 * note_t))
        
        # ADSR エンベロープ
        envelope = np.ones_like(note_t)
        attack_samples = int(0.05 * sample_rate)  # 50ms attack
        release_samples = int(0.1 * sample_rate)  # 100ms release
        
        if len(envelope) > attack_samples:
            envelope[:attack_samples] = np.linspace(0, 1, attack_samples)
        if len(envelope) > release_samples:
            envelope[-release_samples:] = np.linspace(1, 0, release_samples)
        
        music_signal[start_idx:end_idx] = note_signal * envelope
    
    return t_total, music_signal, notes

# 和音の生成
def create_chord_progression(sample_rate):
    """
    和音進行を生成
    """
    # C - Am - F - G progression
    chords = [
        [261.63, 329.63, 392.00],  # C major (C-E-G)
        [220.00, 261.63, 329.63],  # A minor (A-C-E)
        [174.61, 220.00, 261.63],  # F major (F-A-C)
        [196.00, 246.94, 293.66]   # G major (G-B-D)
    ]
    
    chord_duration = 1.0  # 1秒
    total_duration = len(chords) * chord_duration
    t_total = np.linspace(0, total_duration, int(sample_rate * total_duration), False)
    
    chord_signal = np.zeros_like(t_total)
    
    for i, chord in enumerate(chords):
        start_idx = int(i * chord_duration * sample_rate)
        end_idx = int((i + 1) * chord_duration * sample_rate)
        
        chord_t = t_total[start_idx:end_idx] - i * chord_duration
        
        # 和音の各音を合成
        chord_sound = np.zeros_like(chord_t)
        for freq in chord:
            chord_sound += np.sin(2 * np.pi * freq * chord_t)
        
        chord_signal[start_idx:end_idx] = chord_sound / len(chord)  # 正規化
    
    return t_total, chord_signal, chords

# 音楽信号の生成
t_music, music_signal, notes = create_musical_sequence(sample_rate, 0.4)
t_chord, chord_signal, chords = create_chord_progression(sample_rate)

# STFTパラメータ
nperseg = 512
noverlap = nperseg // 2

# メロディーのSTFT
f_music, t_stft_music, Zxx_music = signal.stft(music_signal, sample_rate, 
                                              nperseg=nperseg, noverlap=noverlap)

# 和音のSTFT
f_chord, t_stft_chord, Zxx_chord = signal.stft(chord_signal, sample_rate,
                                              nperseg=nperseg, noverlap=noverlap)

plt.figure(figsize=(16, 12))

# メロディー - 時間波形
plt.subplot(3, 2, 1)
plt.plot(t_music, music_signal)
plt.title('メロディー（C Major Scale）')
plt.ylabel('振幅')
plt.grid(True)

# メロディー - スペクトログラム
plt.subplot(3, 2, 2)
plt.pcolormesh(t_stft_music, f_music, 20 * np.log10(np.abs(Zxx_music) + 1e-10),
              shading='gouraud', cmap='viridis')
plt.title('メロディーのスペクトログラム')
plt.ylabel('周波数 (Hz)')
plt.colorbar(label='振幅 (dB)')
plt.ylim(0, 1000)

# 理論値（ノートの周波数）をプロット
for i, note_freq in enumerate(notes):
    note_time = i * 0.4 + 0.2  # 各ノートの中央時刻
    plt.plot(note_time, note_freq, 'wo', markersize=8, markeredgecolor='red')
    plt.text(note_time, note_freq + 50, f'{note_freq:.0f}', 
            ha='center', color='white', fontsize=8)

# 和音 - 時間波形
plt.subplot(3, 2, 3)
plt.plot(t_chord, chord_signal)
plt.title('和音進行（C-Am-F-G）')
plt.ylabel('振幅')
plt.grid(True)

# 和音 - スペクトログラム
plt.subplot(3, 2, 4)
plt.pcolormesh(t_stft_chord, f_chord, 20 * np.log10(np.abs(Zxx_chord) + 1e-10),
              shading='gouraud', cmap='plasma')
plt.title('和音のスペクトログラム')
plt.ylabel('周波数 (Hz)')
plt.colorbar(label='振幅 (dB)')
plt.ylim(0, 800)

# 理論値（和音の構成音）をプロット
for i, chord in enumerate(chords):
    chord_time = i * 1.0 + 0.5  # 各和音の中央時刻
    for freq in chord:
        plt.plot(chord_time, freq, 'wo', markersize=6, markeredgecolor='red')

# 周波数解析（特定時刻でのスペクトル）
plt.subplot(3, 2, 5)
# メロディーの3番目のノート（ミ）の時刻
note_time_idx = np.argmin(np.abs(t_stft_music - 1.0))  # 約1秒
spectrum_note = np.abs(Zxx_music[:, note_time_idx])
plt.plot(f_music, 20 * np.log10(spectrum_note + 1e-10), 'b-', linewidth=2, label='ミ（329.63 Hz）')
plt.axvline(x=329.63, color='r', linestyle='--', label='基音')
plt.axvline(x=329.63*2, color='r', linestyle=':', alpha=0.7, label='2倍音')
plt.axvline(x=329.63*3, color='r', linestyle=':', alpha=0.7, label='3倍音')
plt.title('単一ノートのスペクトル（t=1.0s）')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅 (dB)')
plt.legend()
plt.grid(True)
plt.xlim(0, 1500)

# 和音解析（特定時刻でのスペクトル）
plt.subplot(3, 2, 6)
# 最初の和音（C major）の時刻
chord_time_idx = np.argmin(np.abs(t_stft_chord - 0.5))  # 約0.5秒
spectrum_chord = np.abs(Zxx_chord[:, chord_time_idx])
plt.plot(f_chord, 20 * np.log10(spectrum_chord + 1e-10), 'g-', linewidth=2, label='C major')

# C major和音の構成音をマーク
c_major = [261.63, 329.63, 392.00]
for freq in c_major:
    plt.axvline(x=freq, color='r', linestyle='--', alpha=0.7)
    plt.text(freq, max(20 * np.log10(spectrum_chord + 1e-10)) - 10, 
            f'{freq:.0f}', rotation=90, ha='center', fontsize=9)

plt.title('和音のスペクトル（t=0.5s, C major）')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅 (dB)')
plt.grid(True)
plt.xlim(0, 800)

plt.tight_layout()
plt.show()

## 3. STFTの逆変換

In [ ]:
# STFT逆変換の実演
def stft_filtering_demo():
    """
    STFTを使った時間-周波数領域でのフィルタリングデモ
    """
    # 複雑な信号の生成（音楽 + ノイズ）
    duration = 4.0
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    
    # 基本信号（メロディー）
    melody_freqs = [261.63, 293.66, 329.63, 349.23]  # C-D-E-F
    melody = np.zeros_like(t)
    
    for i, freq in enumerate(melody_freqs):
        start_time = i
        end_time = i + 1
        mask = (t >= start_time) & (t < end_time)
        melody[mask] = np.sin(2 * np.pi * freq * t[mask])
    
    # ノイズを追加
    np.random.seed(42)
    noise = 0.5 * np.random.normal(0, 1, len(t))
    
    # 妨害信号（高周波数）
    interference = 0.3 * np.sin(2 * np.pi * 1500 * t) * (np.sin(2 * np.pi * 2 * t) > 0)
    
    # 合成信号
    noisy_signal = melody + noise + interference
    
    # STFT
    f, t_stft, Zxx_noisy = signal.stft(noisy_signal, sample_rate, 
                                      nperseg=512, noverlap=256)
    
    # フィルタリング（時間-周波数マスク）
    # 1. ローパスフィルタ（1000Hz以下を保持）
    lowpass_mask = f <= 1000
    Zxx_lowpass = Zxx_noisy.copy()
    Zxx_lowpass[~lowpass_mask, :] = 0
    
    # 2. ノイズゲート（振幅が小さい成分を除去）
    magnitude = np.abs(Zxx_noisy)
    threshold = np.percentile(magnitude, 70)  # 70パーセンタイル以下を除去
    noise_gate_mask = magnitude > threshold
    Zxx_gated = Zxx_noisy * noise_gate_mask
    
    # 3. 時間選択フィルタ（特定時間帯のみ保持）
    time_mask = (t_stft >= 1.0) & (t_stft <= 3.0)  # 1-3秒
    Zxx_time_filtered = Zxx_noisy.copy()
    Zxx_time_filtered[:, ~time_mask] = 0
    
    # 逆STFT
    _, cleaned_lowpass = signal.istft(Zxx_lowpass, sample_rate, nperseg=512, noverlap=256)
    _, cleaned_gated = signal.istft(Zxx_gated, sample_rate, nperseg=512, noverlap=256)
    _, cleaned_time = signal.istft(Zxx_time_filtered, sample_rate, nperseg=512, noverlap=256)
    
    return (t, melody, noisy_signal, cleaned_lowpass, cleaned_gated, cleaned_time,
           f, t_stft, Zxx_noisy, Zxx_lowpass, Zxx_gated, Zxx_time_filtered)

# フィルタリングデモを実行
results = stft_filtering_demo()
(t, melody, noisy_signal, cleaned_lowpass, cleaned_gated, cleaned_time,
 f, t_stft, Zxx_noisy, Zxx_lowpass, Zxx_gated, Zxx_time_filtered) = results

plt.figure(figsize=(18, 15))

# 時間領域の比較
signals = [melody, noisy_signal, cleaned_lowpass, cleaned_gated, cleaned_time]
signal_names = ['元のメロディー', 'ノイズ付き信号', 'ローパスフィルタ後', 'ノイズゲート後', '時間フィルタ後']

for i, (sig, name) in enumerate(zip(signals, signal_names)):
    plt.subplot(4, 5, i + 1)
    plt.plot(t[:len(sig)], sig, linewidth=2)
    plt.title(name)
    plt.ylabel('振幅')
    plt.grid(True)
    if i == 4:
        plt.xlabel('時間 (秒)')

# スペクトログラムの比較
spectrograms = [Zxx_noisy, Zxx_lowpass, Zxx_gated, Zxx_time_filtered]
spectrogram_names = ['元のスペクトログラム', 'ローパスフィルタ', 'ノイズゲート', '時間フィルタ']

for i, (Zxx, name) in enumerate(zip(spectrograms, spectrogram_names)):
    plt.subplot(4, 5, i + 6)
    plt.pcolormesh(t_stft, f, 20 * np.log10(np.abs(Zxx) + 1e-10),
                  shading='gouraud', cmap='viridis')
    plt.title(name)
    plt.ylabel('周波数 (Hz)')
    plt.ylim(0, 2000)
    if i == 3:
        plt.xlabel('時間 (秒)')

# フィルタマスクの可視化
plt.subplot(4, 5, 11)
lowpass_mask = f <= 1000
mask_2d = np.outer(lowpass_mask.astype(float), np.ones(len(t_stft)))
plt.pcolormesh(t_stft, f, mask_2d, shading='gouraud', cmap='RdYlBu')
plt.title('ローパスマスク')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 2000)

plt.subplot(4, 5, 12)
magnitude = np.abs(Zxx_noisy)
threshold = np.percentile(magnitude, 70)
gate_mask = (magnitude > threshold).astype(float)
plt.pcolormesh(t_stft, f, gate_mask, shading='gouraud', cmap='RdYlBu')
plt.title('ノイズゲートマスク')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 2000)

plt.subplot(4, 5, 13)
time_mask = (t_stft >= 1.0) & (t_stft <= 3.0)
time_mask_2d = np.outer(np.ones(len(f)), time_mask.astype(float))
plt.pcolormesh(t_stft, f, time_mask_2d, shading='gouraud', cmap='RdYlBu')
plt.title('時間マスク')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 2000)

# エラー解析
plt.subplot(4, 5, 15)
errors = [
    np.mean((melody - noisy_signal[:len(melody)])**2),
    np.mean((melody - cleaned_lowpass[:len(melody)])**2),
    np.mean((melody - cleaned_gated[:len(melody)])**2),
    np.mean((melody - cleaned_time[:len(melody)])**2)
]
method_names = ['ノイズあり', 'ローパス', 'ノイズゲート', '時間フィルタ']
colors = ['red', 'blue', 'green', 'orange']

bars = plt.bar(method_names, errors, color=colors, alpha=0.7)
plt.title('復元誤差（MSE）')
plt.ylabel('平均二乗誤差')
plt.xticks(rotation=45)
plt.yscale('log')

for bar, error in zip(bars, errors):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 1.5,
            f'{error:.2e}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("STFT逆変換の応用:")
print("1. 時間-周波数領域での選択的フィルタリング")
print("2. ノイズ除去")
print("3. 音響効果処理")
print("4. 音楽の分離・抽出")

## 練習問題

1. 異なる窓関数（Hanning, Hamming, Blackman）がSTFTに与える影響を比較してみましょう
2. 楽器音を録音してSTFT解析し、アタック・サスティン・リリースの特徴を観察してみましょう
3. STFTを使って特定の周波数成分だけを抽出する音響効果を作ってみましょう